In [1]:
from pyspark.ml.feature import (RegexTokenizer, Tokenizer, HashingTF, IDF,
                                StopWordsRemover, CountVectorizer, StopWordsRemover, StringIndexer, OneHotEncoder)
from pyspark.ml.evaluation import (BinaryClassificationEvaluator,
                                  MulticlassClassificationEvaluator)
from pyspark.sql.types import (LongType ,StringType, IntegerType,
                               FloatType, DoubleType, ArrayType)
from pyspark.sql.functions import col, udf, avg
from pyspark.sql.types import IntegerType
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from sklearn.model_selection import train_test_split
from pyspark.ml.clustering import LDA
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import countDistinct
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql import HiveContext
from pyspark.sql import functions as F

In [2]:
#FOR ENSEMBLE EXPERIMENT ONLY!!!
df_doiCateg = spark.sql("SELECT * FROM taxiarchis.doi_categories_4_4_features")
df = spark.sql("SELECT * FROM taxiarchis.section_4_4_ensemble").dropna()

indexer = StringIndexer(inputCol="Category", outputCol="label")
indexed = indexer.fit(df_doiCateg).transform(df_doiCateg)

#transfrom to pandas
df_cat = indexed.toPandas()

# #TRAINING-VALIDATION (0.4-0.6)
# #Split the DOIs for training and Test-----random state =42

# X_train, X_validation, y_train, y_validation  = train_test_split(df_cat['DOI'],df_cat['label'], test_size=0.6, random_state=42)


# #KEEP IT FOR VECTORIZE LATER
# second_split_back_up = X_validation



# #VALIDATION-TEST (0.66-0.34)--->(0.4,0.2) from original dataset
# X_validation, X_test, y_validation, y_test = train_test_split(X_validation, y_validation, test_size=0.34, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(df_cat['DOI'],df_cat['label'], test_size=0.30, random_state=42)
second_split_back_up = X_train

print(len(X_train))
#print(len(X_validation))
print(len(X_test))
print(len(second_split_back_up))

# TF-IDF

In [4]:
regexTokenizer = RegexTokenizer(inputCol="Section", outputCol="tokens", pattern="\\W")
remover = StopWordsRemover(inputCol='tokens', outputCol='filtered')
count_vec = CountVectorizer(inputCol='filtered', outputCol='count_vec')
idf = IDF(inputCol='count_vec', outputCol='features',minDocFreq=2)

#STRINGINDEXER THE DOC_ID

int_category = StringIndexer(inputCol='Category',outputCol='label')
int_sections = StringIndexer(inputCol = 'doc_id', outputCol ='uni_sec_id')


prep_pipeline = Pipeline(stages = [int_category, int_sections, regexTokenizer, remover, count_vec, idf])
pre_processing = prep_pipeline.fit(df)
pre_processed_data = pre_processing.transform(df)

final_data_2 = pre_processed_data.select(['doc_id','uni_sec_id','DOI','label','features']).cache()
# display(final_data)
final_data_2.count()

In [5]:
regexTokenizer = RegexTokenizer(inputCol="Section", outputCol="tokens", pattern="\\W")
remover = StopWordsRemover(inputCol='tokens', outputCol='filtered')
count_vec = CountVectorizer(inputCol='filtered', outputCol='count_vec')
idf = IDF(inputCol='count_vec', outputCol='features',minDocFreq=2)

#STRINGINDEXER THE DOC_ID

int_category = StringIndexer(inputCol='Category',outputCol='label')
int_sections = StringIndexer(inputCol = 'doc_id', outputCol ='uni_sec_id')


prep_pipeline = Pipeline(stages = [int_category, int_sections, regexTokenizer, remover, count_vec, idf])
pre_processing = prep_pipeline.fit(df)
pre_processed_data = pre_processing.transform(df)

final_data = pre_processed_data.select(['doc_id','uni_sec_id','DOI','label','features']).cache()
# display(final_data)
final_data.count()

# LDA

In [7]:
from pyspark.ml.clustering import LDA

regexTokenizer = RegexTokenizer(inputCol='Section', outputCol="tokens", pattern="\\W")
remover = StopWordsRemover(inputCol='tokens', outputCol='filtered')
count_vec = CountVectorizer(inputCol='filtered', outputCol='features')

int_category = StringIndexer(inputCol='Category',outputCol='label')
int_id = StringIndexer(inputCol='doc_id',outputCol='id') #map each unique document with an id
#PIPELINE


prep_pipeline = Pipeline(stages = [int_category, int_id, regexTokenizer, remover, count_vec])

pre_processing = prep_pipeline.fit(df)
pre_processed_data = pre_processing.transform(df)

final_data_LDA = pre_processed_data.cache()



final_data_LDA = final_data_LDA.dropDuplicates(['id'])
# final_data_LDA = final_data_LDA.select(['id','DOI','contENT','features','Section','Category','Sections','label'])
# final_data_LDA.count()

# Trains a LDA model.
lda = LDA(k=30, maxIter=50, optimizer="em")
model = lda.fit(final_data_LDA.select('id','features'))


#Create LDA features
transformed = model.transform(final_data_LDA)
final_data = transformed.select(['DOI','label','topicDistribution']).withColumnRenamed("topicDistribution", "features").cache()



transformed_2 = model.transform(final_data_LDA)
final_data_2 = transformed_2.select(['DOI','label','topicDistribution']).withColumnRenamed("topicDistribution", "features").cache()


transformed_3 = model.transform(final_data_LDA)
final_data_3 = transformed_3.select(['DOI','label','topicDistribution']).withColumnRenamed("topicDistribution", "features").cache()

In [8]:
from pyspark.ml.clustering import LDA

regexTokenizer = RegexTokenizer(inputCol='Section', outputCol="tokens", pattern="\\W")
remover = StopWordsRemover(inputCol='tokens', outputCol='filtered')
count_vec = CountVectorizer(inputCol='filtered', outputCol='features')

int_category = StringIndexer(inputCol='Category',outputCol='label')
int_id = StringIndexer(inputCol='doc_id',outputCol='id') #map each unique document with an id
#PIPELINE


prep_pipeline = Pipeline(stages = [int_category, int_id, regexTokenizer, remover, count_vec])

pre_processing = prep_pipeline.fit(df)
pre_processed_data = pre_processing.transform(df)

final_data_LDA = pre_processed_data.cache()



final_data_LDA = final_data_LDA.dropDuplicates(['id'])
# final_data_LDA = final_data_LDA.select(['id','DOI','contENT','features','Section','Category','Sections','label'])
# final_data_LDA.count()

# Trains a LDA model.
lda = LDA(k=100, maxIter=50, optimizer="em")
model = lda.fit(final_data_LDA.select('id','features'))


#Create LDA features
transformed = model.transform(final_data_LDA)
final_data = transformed.select(['DOI','label','topicDistribution']).withColumnRenamed("topicDistribution", "features").cache()
final_data.show()

# Classification Algorithms

In [10]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import (RandomForestClassifier, GBTClassifier,
                                      DecisionTreeClassifier)
from pyspark.ml.classification import LinearSVC, OneVsRest
from pyspark.ml.classification import LogisticRegression

#Multinomial Naive Bayes
nb = NaiveBayes()

#Decision Tree - Random Forest 
dtc = DecisionTreeClassifier(maxDepth = 12)
# maxDepth = 15
rfc = RandomForestClassifier(numTrees = 200, maxDepth = 20)

#Linear SVM - One vs All
lsvc = LinearSVC(maxIter=10, regParam=0.2)
ovr = OneVsRest(classifier=lsvc  )

#Logistic Regression
lr = LogisticRegression(maxIter=15, regParam=0.0, elasticNetParam=0.0 )



# SPLIT TRAINING-VALIDATION

In [12]:
#train
train = final_data.filter(final_data.DOI.isin(list(X_train)))

# #validation
# validation = final_data.filter(final_data.DOI.isin(list(X_validation)))

#validation
validation = final_data.filter(final_data.DOI.isin(list(X_test)))
#For strong classifier Ensemble
sec_classifir_vec  = final_data.filter(final_data.DOI.isin(list(second_split_back_up)))

print(train.count())
print(validation.count())
print(sec_classifir_vec.count())

# For the LDA representation

In [14]:
#train
train_2 = final_data_2.filter(final_data_2.DOI.isin(list(X_train)))

# #validation
# validation = final_data.filter(final_data.DOI.isin(list(X_validation)))

#validation
validation_2 = final_data_2.filter(final_data_2.DOI.isin(list(X_test)))
#For strong classifier Ensemble
sec_classifir_vec_2  = final_data_2.filter(final_data_2.DOI.isin(list(second_split_back_up)))

print(train_2.count())
print(validation_2.count())
print(sec_classifir_vec_2.count())

In [15]:
#train
train_3 = final_data_3.filter(final_data_3.DOI.isin(list(X_train)))

# #validation
# validation = final_data.filter(final_data.DOI.isin(list(X_validation)))

#validation
validation_3 = final_data_3.filter(final_data_3.DOI.isin(list(X_test)))
#For strong classifier Ensemble
sec_classifir_vec_3  = final_data_3.filter(final_data_3.DOI.isin(list(second_split_back_up)))

print(train_3.count())
print(validation_3.count())
print(sec_classifir_vec_3.count())

# Evaluation Function

In [17]:
def evaluation_measures(new_df_p):
    '''
    Predict all the evaluation Measures:
    Accuracy, F1-Score, Recall, Precision
    '''
    
    ev_m = []
    
    #ACCURACY
    acc_eval = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",metricName = 'accuracy')
    accuracy = acc_eval.evaluate(new_df_p)
    print("Accuracy = %g" % accuracy)
    ev_m.append(accuracy)
    
    #F1-SCORE
    acc_eval_f1 = MulticlassClassificationEvaluator(metricName = 'f1')
    f1_score = acc_eval_f1.evaluate(new_df_p)
    print("f1 = %g" % f1_score)
    ev_m.append(f1_score)
    
    #RECALL
    acc_eval_recall = MulticlassClassificationEvaluator(metricName = 'weightedRecall')
    recall = acc_eval_recall.evaluate(new_df_p)
    print("weightedRecall = %g" % recall)
    ev_m.append(recall)
    
    #PRECISION
    acc_eval_precission = MulticlassClassificationEvaluator(metricName = "weightedPrecision")
    precission = acc_eval_precission.evaluate(new_df_p)
    print("weightedPrecision = %g" % precission)
    ev_m.append(precission)
    
    return ev_m

# Train - Evaluate - Transform for the second classifier

In [19]:
#train classifier - train set
paper_class = lr.fit(train) #
paper_class_2 = dtc.fit(train_2)
paper_class_3 = rfc.fit(train_3)
#Prepare features for the second Classifier
# df_Second = paper_class.transform(sec_classifir_vec)



df_Second = paper_class.transform(final_data)
df_Second_2 = paper_class_2.transform(final_data_2)
df_Second_3 = paper_class_3.transform(final_data_3)


df_Second_train = paper_class.transform(sec_classifir_vec)
df_Second_2_train = paper_class_2.transform(sec_classifir_vec_2)
df_Second_3_train = paper_class_3.transform(sec_classifir_vec_3)



#predict labels - validation set
# validation_results = paper_class.transform(validation)
# evaluation_measures(validation_results)

print(df_Second.count())
print(df_Second_2.count())
print(df_Second_3.count())
print(df_Second_train.count())
print(df_Second_2_train.count())
print(df_Second_3_train.count())

In [20]:
df_Second.show()

In [21]:
def find_maximum(test_results):
    '''
    Finds the maximum probability and 
    prepares the dataframe for the Majority Vote.
    '''  
    udf_wf_var = udf(lambda x: round(np.max(np.array(x)),10), returnType=FloatType()) #Define UDF function
    df =  test_results.withColumn('WF_Var',udf_wf_var('probability')).dropna()

    return df.groupby("DOI").agg(F.collect_list('prediction').alias('list_categ'),F.collect_list('WF_Var').alias('prob_categ'),F.collect_list('doc_id').alias('Sec_ids'))

test_df_sect = find_maximum(validation_results)
display(test_df_sect)

# Prepare Vectorized Dataframe for the Second Classifier

In [23]:
import numpy as np
from pyspark.sql.types import (LongType ,StringType, IntegerType,
                               FloatType, DoubleType, ArrayType)
from pyspark.sql.functions import countDistinct

##TEST SECOND SCENARIO


def find_maximum_3(test_results):
    '''
    Finds the maximum probability and 
    prepares the dataframe for the Majority Vote.
    '''  
    udf_wf_var = udf(lambda x: round(np.max(np.array(x)),10), returnType=FloatType()) #Define UDF function
    df =  test_results.withColumn('WF_Var',udf_wf_var('probability')).dropna()

    return df.groupby("DOI").agg(F.collect_list('prediction').alias('list_categ'),F.collect_list('WF_Var').alias('prob_categ_3'))


def find_maximum_2(test_results):
    '''
    Finds the maximum probability and 
    prepares the dataframe for the Majority Vote.
    '''  
    udf_wf_var = udf(lambda x: round(np.max(np.array(x)),10), returnType=FloatType()) #Define UDF function
    df =  test_results.withColumn('WF_Var',udf_wf_var('probability')).dropna()

    return df.groupby("DOI").agg(F.collect_list('prediction').alias('list_categ'),F.collect_list('WF_Var').alias('prob_categ_2'))



def find_maximum(test_results):
    '''
    Finds the maximum probability and 
    prepares the dataframe for the Majority Vote.
    '''  
    udf_wf_var = udf(lambda x: round(np.max(np.array(x)),10), returnType=FloatType()) #Define UDF function
    df =  test_results.withColumn('WF_Var',udf_wf_var('probability')).dropna()

    return df.groupby("DOI").agg(F.collect_list('prediction').alias('list_categ'),F.collect_list('WF_Var').alias('prob_categ'))


def filter_arrays(ar):
    '''
    Returns only the arrays that match with 
    the array_length.
    '''
    array_length = 4
    if len(ar)!=array_length:
          return int(0)
    else:
          return int(1)
    

def clear_corrupt_vectors(new_set):
    '''
    Filter out corrupt or unmatching 
    vector lengths by using the filter_arrays
    function. The function has a fixed length of 4
    need to be changed if needs to be tested higher vectors.
    '''
    flt = udf( filter_arrays,IntegerType())
    filt_df =new_set.withColumn('binary_filter',flt(new_set.prob_categ))

    return filt_df.filter(col('binary_filter').isin([1]))

def clear_corrupt_vectors_2(new_set):
    '''
    Filter out corrupt or unmatching 
    vector lengths by using the filter_arrays
    function. The function has a fixed length of 4
    need to be changed if needs to be tested higher vectors.
    '''
    flt = udf( filter_arrays,IntegerType())
    filt_df =new_set.withColumn('binary_filter',flt(new_set.prob_categ_2))

    return filt_df.filter(col('binary_filter').isin([1]))

def clear_corrupt_vectors_3(new_set):
    '''
    Filter out corrupt or unmatching 
    vector lengths by using the filter_arrays
    function. The function has a fixed length of 4
    need to be changed if needs to be tested higher vectors.
    '''
    flt = udf( filter_arrays,IntegerType())
    filt_df =new_set.withColumn('binary_filter',flt(new_set.prob_categ_3))

    return filt_df.filter(col('binary_filter').isin([1]))

def vectorize_array(df):
    '''
    Vectorize a dataframe's column with arrays.
    Preparing the dataframe for the classifier.
    '''
    to_vector = udf(lambda a: Vectors.dense(a), VectorUDT())
    
    return df.select('DOI', to_vector("prob_categ").alias("features_1"))


def vectorize_array_2(df):
    '''
    Vectorize a dataframe's column with arrays.
    Preparing the dataframe for the classifier.
    '''
    to_vector = udf(lambda a: Vectors.dense(a), VectorUDT())
    
    return df.select('DOI', to_vector("prob_categ_2").alias("features_2"))
  
def vectorize_array_3(df):
    '''
    Vectorize a dataframe's column with arrays.
    Preparing the dataframe for the classifier.
    '''
    to_vector = udf(lambda a: Vectors.dense(a), VectorUDT())
    
    return df.select('DOI', to_vector("prob_categ_3").alias("features_3"))
  
def prepare_for_strong_classifier(test_results):
    '''
    Uses all the above functions to transform
    '''
    
    #Find Maximum Probability and prepare dataframe for majority vote
    df_prep = find_maximum(test_results)

    #prepare the dataset to send to the stronger classifier
    return vectorize_array(clear_corrupt_vectors(df_prep))
  
#SECOND TRY 
  
def prepare_for_strong_classifier_2(test_results):
    '''
    Uses all the above functions to transform
    '''
    
    #Find Maximum Probability and prepare dataframe for majority vote
    df_prep = find_maximum_2(test_results)

    #prepare the dataset to send to the stronger classifier
    return vectorize_array_2(clear_corrupt_vectors_2(df_prep))
  
def prepare_for_strong_classifier_3(test_results):
    '''
    Uses all the above functions to transform
    '''
    
    #Find Maximum Probability and prepare dataframe for majority vote
    df_prep = find_maximum_3(test_results)

    #prepare the dataset to send to the stronger classifier
    return vectorize_array_3(clear_corrupt_vectors_3(df_prep))
    

#Decision Tree
df_train_2_1 = prepare_for_strong_classifier(df_Second)#TOD-DO CHECK IF THE COLUMSN ARE RIGHT ADN IS THE CORRECT DATAFRAME!!

#Logistic Regresssion
df_train_2_2 = prepare_for_strong_classifier_2(df_Second_2)

df_train_2_3 = prepare_for_strong_classifier_3(df_Second_3)
#FIRST MAKE THE VECTOR aSSEMBLER!!!! THEN MERGE FOR THE LABELS

print(df_train_2_1.count())
print(df_train_2_2.count())
print(df_train_2_3.count())


# Inner Merge with real Categories
# df_train_2 = df_train_2.join(indexed , on=['DOI'], how='inner').dropDuplicates().dropna().cache()



# display(df_train_2)


In [24]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ['features_1', 'features_2','features_3'], outputCol='features')
output = assembler.transform(df_train_test)
display(output)

In [25]:
output.count()

In [26]:
# Inner Merge with real Categories
df_train_final = output.select(['DOI', 'features']).join(indexed , on=['DOI'], how='inner').dropDuplicates().dropna().cache()
display(df_train_final)

In [27]:
#train
train = df_train_final.filter(df_train_final.DOI.isin(list(X_train)))

# #validation
# validation = final_data.filter(final_data.DOI.isin(list(X_validation)))

#validation
validation = df_train_final.filter(df_train_final.DOI.isin(list(X_test)))
# #For strong classifier Ensemble
# sec_classifir_vec  = final_data.filter(final_data.DOI.isin(list(second_split_back_up)))

print(train.count())
print(validation.count())

In [28]:
paper_class_2 = ovr.fit(train)
df_Second_results = paper_class_2.transform(validation)
evaluation_measures(df_Second_results)

In [29]:
paper_class_2 = rfc.fit(train)
df_Second_results = paper_class_2.transform(validation)
evaluation_measures(df_Second_results)

In [30]:
#DONT DO IT FOR THE SECOND EXPERIMENT

#validation
validation_2 = df_train_2.filter(df_train_2.DOI.isin(list(X_validation)))

#test
test_df_f = df_train_2.filter(df_train_2.DOI.isin(list(X_test)))

print(validation_2.count())
print(test_df_f.count())

In [31]:
validation_2.groupBy('Category').count().show()

In [32]:
test_df_f.groupBy('Category').count().show()

In [33]:
df_train_2.count()

In [34]:



paper_class_2 = rfc.fit(df_train_2)
df_Second_results = paper_class_2.transform(validation)
# evaluation_measures(df_Second_results)

In [35]:
df_Second_results.count()

In [36]:
dict_algo={'nb':'Multinomial Naive Bayes',
           'lr':'Multinomial Logistic Regression',
           'dtc':'Decision Tree Classifier',
           'rfc':'Random Forest Classifier',
           'ovr':'One vs Rest Linear SVM'
}


algorithms = [dtc, nb,lr,ovr,rfc]
names = ["dtc", "nb", 'lr', "ovr",'rfc']
for algo, n in zip(algorithms,names):
    print(dict_algo[n])
    paper_class_2 = algo.fit(validation_2)
    df_Second_results = paper_class_2.transform(test_df_f)
    evaluation_measures(df_Second_results)

In [37]:
#train classifier - train set
paper_class_2 = nb.fit(validation_2)

#predict labels - test set
df_Second_results = paper_class_2.transform(test)
# display(df_Second_results)
#EVALUATE
evaluation_measures(df_Second_results)

##Simple majority vote

In [40]:
from pyspark import SparkContext
from pyspark.sql import HiveContext
from pyspark.sql import functions as F
from pyspark.sql import functions as F
import numpy as np
import random 
from pyspark.sql.types import IntegerType,DoubleType
from pyspark.sql.functions import col


def prepare_rand_majority(df):
  '''
  Return a dataframe which contains
  in each row a document and a list of
  the predictions of the classifier prepared 
  for the majority vote.
  '''
  return  df.groupby("DOI").agg(F.collect_list("prediction").alias('list_categ'))


def majority_vote_random(votes):
    '''
    Predict the majority vote by
    returning randomly when we have more
    than one winner class.
    '''

    vote_counts = {}
    
    for vote in votes:
        if vote in vote_counts:
            vote_counts[vote] += 1
        else:
            vote_counts[vote] = 1
    winners = []
    max_count = np.max(vote_counts.values())
    
    for vote, count in vote_counts.items():
        if count == max_count:
            winners.append(vote)
            
    return random.choice(winners)

  
def final_prediction_maj_random(df):
    '''
    Computes the final predictions based
    on a simple majority vote.
    '''
    df_for_majority_vote = prepare_rand_majority(df)
    #udf apply function
    majority_function = udf(lambda z: majority_vote_random(z), DoubleType())
#     final_prediction = df_for_majority_vote.withColumn('prediction', majority_function(col('list_categ')))
    f_res = df_for_majority_vote.withColumn('prediction', majority_function(col('list_categ')))
    return f_res.join(indexed , on=['DOI'], how='inner').dropDuplicates().dropna().cache()